In [2]:
datadir = "D:\ReactOS\\ROS-VS2012-Debug-x86\\reactos\\output-VS-i386\\"
#datadir = "D:\ReactOS\\ROS-VS2015-Release-x86\\reactos\\output-VS-i386\\"
stageDir = "D:\\staging\\"
datasetDir = "D:\\datasets\\reactos\\"

exeTypes = [".exe",".dll",".sys"]

In [3]:
pes = []
allFiles = []

#datadir ="d:\\staging\\"

for root, dir, files in os.walk(datadir):
    for f in files:
        allFiles.append(root+"\\"+f)
        
allFiles = list(set(allFiles))

for f in tqdm(allFiles):
    c = 0
    hasPDB = False
    exSec = False
    try:
        pe = pefile.PE(f)
    except:
        continue
    for s in pe.sections:
        if s.IMAGE_SCN_MEM_EXECUTE:
            c = c + 1
            exSec = True
    buff = open(f,"rb").read()
    pdbstr = re.findall("D:.*\.pdb",buff)
    if pdbstr:
        hasPDB = True
    if hasPDB and exSec:
        pes.append(f)
print len(pes)
            

100%|████████████████████████████████████████████████████████████████████████████| 35454/35454 [06:00<00:00, 98.44it/s]

673



100%|████████████████████████████████████████████████████████████████████████████| 35454/35454 [06:00<00:00, 98.44it/s]

673


673


In [4]:
           
for f in tqdm(pes):
    baseName = f.split("\\")[-1]
    shutil.copyfile(f,stageDir+baseName)
    
    

100%|████████████████████████████████████████████████████████████████████████████████| 673/673 [00:09<00:00, 68.37it/s]


In [5]:
def processPDB(filename, codeOffset):
    d = {}
    p = open("C:\\Temp\\pdbtmp.txt","w")
    sp.call(["c:\\tools\\dia2dump.exe","-s",filename], stdout=p)
    p.close()
    p = open("C:\\Temp\\pdbtmp.txt","r")
    for line in p:
        if re.match("Function",line):
            tok = line.split(",")
            #print tok
            name = tok[3].strip()
            length = tok[2].split("=")[1]
            offset = tok[1].split("]")[0].split("[")[1]
            d[name] = (int(offset,16)-codeOffset,int(length,16))
    p.close()
    
    return d

def getMD5(filename):
    hasher = hashlib.md5()
    with open(filename, 'rb') as afile:
        buf = afile.read()
        hasher.update(buf)
    afile.close
    return hasher.hexdigest()

def getImports(pe):
    imports = {}
    for entry in pe.DIRECTORY_ENTRY_IMPORT:
        for imp in entry.imports:
            cname = entry.dll+"!"+imp.name
            #Need to AND cname because murmur hash returns signed int
            imports[hex(imp.address)] = (cname, mmh3.hash(cname) & 0xffffffff)
    return imports

def normalizeCode(rawCode, codeBase, imports):  
    md = Cs(CS_ARCH_X86, CS_MODE_32)
    md.detail = True
    
    normCode = rawCode
    
    for insn in md.disasm(rawCode, codeBase):
        offset = insn.address - codeBase
        end = offset + insn.size
        if insn.mnemonic == "call":
            addrs = re.findall(r'0x[0-9a-fA-F]+', insn.op_str)
            for a in addrs:
                if a in imports:
                    ibytes = ""
                    ihash = str(hex(imports[a][1])[2:])
                    #Pad hash with 0 
                    if len(ihash) < 8:
                        ihash = ''.join(["0" for x in xrange(0,8-len(ihash))]) + ihash
                    for i in xrange(0, 8, 2):
                        ibytes += chr(int(ihash[i:i+2],16))
                    normCode = rawCode[:end-3] + ibytes + rawCode[end+1:]
    return normCode
    
#processPDB("d:\\ReactOS\\ROS-VS2015-Debug-x86\\reactos\\output-VS-i386\\msvc_pdb\\Debug\\write.pdb")

In [6]:
def dictify(filename):
    d = {}
    buff = open(filename,"rb").read()
    PDBPath = re.findall("D:.*\.pdb",buff)
    pe = pefile.PE(filename)
    for s in pe.sections:
        if s.IMAGE_SCN_MEM_EXECUTE:
            codeOffset = s.VirtualAddress
            codeLen = s.SizeOfRawData
    d["Name"] = filename.split("\\")[-1]
    d["MD5"] = getMD5(filename)
    d["Collection"] = "ReactOS"
    d["CC_Name"] = "Visual Studio"
    d["CC_Version"] = "11"
    d["Arch"] = "x86"
    d["Build_Flags"] = "Debug"
    d["CodeBase"] = pe.OPTIONAL_HEADER.ImageBase + pe.OPTIONAL_HEADER.BaseOfCode
    d["Funcs"] = processPDB(PDBPath, codeOffset)
    d["Imports"] = getImports(pe)
    d["Raw_Code"] = base64.b64encode(pe.get_memory_mapped_image()[codeOffset:codeOffset+codeLen])
    d["Normalized_Code"] = base64.b64encode(normalizeCode(d["Raw_Code"], d["CodeBase"], d["Imports"]))
    pe.close()

    return d

#myBlob = dictify("d:\\staging\\write.exe")
#print myBlob

In [7]:
def makeDataSetItem(filename):
    try:
        d = dictify(filename)
    except:
        return
    with open(datasetDir+d["MD5"],"w") as outfile:
        json.dump(d,outfile)

In [8]:
for f in tqdm(os.listdir(stageDir)):
    makeDataSetItem(stageDir+f)

100%|████████████████████████████████████████████████████████████████████████████████| 665/665 [10:50<00:00,  1.13s/it]



In [ ]:
b